In [3]:
from extract_feature import *
from feature_set import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
dataset = read_all_DS()
print("done!")
pa_indexed = dataset['paper_author'].set_index(['AuthorId','PaperId']).sort_index()
author_indexed = dataset['author'].set_index(['Id']).sort_index()
paper_indexed = dataset['paper'].set_index(['Id']).sort_index()


done!


In [7]:
trainset = pd.read_csv('dataRev2/Train.csv')
train_confirmed = trainset[['AuthorId', 'ConfirmedPaperIds']].rename(columns = {'ConfirmedPaperIds':'PaperIds'})
train_deleted = trainset[['AuthorId', 'DeletedPaperIds']].rename(columns = {'DeletedPaperIds':'PaperIds'})
validset = pd.read_csv('dataRev2/Valid.csv')
testset = pd.read_csv('dataRev2/Test.csv')
allsets = pd.concat([train_confirmed, validset, testset])
author_paper_pairs = parse_targetset(train_confirmed)
all_dups = make_duplicates_from_targets(allsets)
dataset['all_duplicates'] = all_dups

In [13]:
#get_ap_pair_from_aid(dataset, 1932365) # professor's author id
target_author_id = 768400
data =get_features_from_aid(dataset, target_author_id)
#print (data)
author_paper_ids = [x[:2] for x in data]
features = [x[2:] for x in data]

print("Loading the classifier")
classifier = data_io.load_model()

print("Making predictions\n")

features = np.array(features)  # This line is for xgboost
predictions = classifier.predict_proba(features)[:, 1]
predictions = list(predictions)

result = []
for (a_id, p_id), pred in zip(author_paper_ids, predictions):
    result.append((pred, p_id))
#print(author_predictions)
paper_ids_sorted = sorted(result, reverse=True)

print("Author name: %s\n" %author_indexed.loc[target_author_id]['Name'])
print("Author affiliation: %s\n" %author_indexed.loc[target_author_id]['Affiliation'])
paper_ids_sorted
for predIdPair in paper_ids_sorted:
    pred, id = predIdPair
    title = paper_indexed.loc[id]['Title']
    print("Id: %s \nPrediction: %s \nTitle: %s\n" % (id, pred, title))

Loading the classifier
Making predictions

Author name: min-soo kim

Author affiliation: dgist

Id: 1281220 
Prediction: 0.99308 
Title: A Logical Model and Data Placement Strategies for MEMS Storage Devices

Id: 1322665 
Prediction: 0.992681 
Title: CHRONICLE: A Two-Stage Density-Based Clustering Algorithm for Dynamic Networks

Id: 1931472 
Prediction: 0.991881 
Title: Structural optimization of a full-text n-gram index using relational normalization

Id: 704880 
Prediction: 0.991762 
Title: Structural optimization of a full-text n -gram index using relational normalization

Id: 319164 
Prediction: 0.991647 
Title: n-Gram/2L: A Space and Time Efficient Two-Level n-Gram Inverted Index Structure

Id: 62278 
Prediction: 0.991547 
Title: Structural Consistency: Enabling XML Keyword Search to Eliminate Spurious Results Consistently

Id: 275269 
Prediction: 0.990377 
Title: A Particle-and-Density Based Evolutionary Clustering Method for Dynamic Networks

Id: 1152394 
Prediction: 0.989964 
T